Must run the following:

In [1]:
!git clone https://github.com/dvamossy/EmTract.git
%cd EmTract
!pip install -r requirements.txt 

D:\Research\FinancialMarkets\Emotions\Emtract\Training\EmTract


Text Cleaner for unprocessed text

In [2]:
from emtract.processors.cleaning import clean_text
# Illustrate text cleaning
clean_text("soooooo well", segment_words=False)

D:\Research\FinancialMarkets\Emotions\Emtract\Training\EmTract\emtract\processors\cleaning.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  symspell_list = pd.read_csv(


'soo well'

Option I

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification", model="vamossyd/emtract-distilbert-base-uncased-emotion", return_all_scores=True)
classifier("i love this!")

C:\Users\Owner\anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'Neutral', 'score': 0.0034543664660304785},
  {'label': 'Happy', 'score': 0.9890496730804443},
  {'label': 'Sad', 'score': 0.0012023887829855084},
  {'label': 'Anger', 'score': 0.0015717606293037534},
  {'label': 'Disgust', 'score': 0.0012307625729590654},
  {'label': 'Surprise', 'score': 0.0021428591571748257},
  {'label': 'Fear', 'score': 0.0013480583438649774}]]

Option II

In [4]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [5]:
input_path = "PROVIDE_PATH_TO_DATA"
# data = pd.read_csv(input_path) # ASSUMING DATA IS IN CSV

# If text is already cleaned:
# texts = data.text.tolist() 

# Otherwise:
# texts = data['text'].apply(clean_text).tolist() # 

# As an example:
texts = ['i love this', 'i do not love you', 'to the moon 🚀']

In [6]:
# load tokenizer and model, create trainer
model_name = "vamossyd/emtract-distilbert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(texts, truncation=True, padding=True)
pred_dataset = SimpleDataset(tokenized_texts)
predictions = trainer.predict(pred_dataset)

In [7]:
# scores raw
temp = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True))
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)

# container
anger = []
disgust = []
fear = []
happy = []
neutral = []
sadness = []
surprise = []

# extract scores (as many entries as exist in pred_texts)
for i in range(len(texts)):
    anger.append(temp[i][3])
    disgust.append(temp[i][4])
    fear.append(temp[i][6])
    happy.append(temp[i][1])
    neutral.append(temp[i][0])
    sadness.append(temp[i][2])
    surprise.append(temp[i][5])
    
df = pd.DataFrame(list(zip(texts, labels, anger, disgust, fear, happy, neutral, sadness, surprise)), columns=['text','pred_label', 'anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])
df.head()

,text,pred_label,anger,disgust,fear,happy,neutral,sad,surprise
0,i love this,Happy,0.001747,0.001476,0.001656,0.989288,0.002194,0.001697,0.001942
1,i do not love you,Sad,0.235049,0.104266,0.116412,0.226152,0.012422,0.269355,0.036345
2,to the moon 🚀,Happy,0.011504,0.010536,0.054058,0.669854,0.201600,0.046615,0.005833


In [ ]:
# save results to csv
output_path = "YOUR_FILENAME_EMOTIONS.csv"  # name your output file
# df.to_csv(YOUR_FILENAME)